In [1]:
import pandas as pd
import eclabfiles as ecf
from matplotlib import pyplot as plt
import numpy as np
import glob
from datetime import datetime
from pickle_utils import read_ecf, read_emsi, optimise_emsi_start
%matplotlib qt

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
DATE = '2024-03-14'

In [4]:
folder = 'T:/Team/Szewczyk/Data/' + DATE
ecfile = glob.glob(folder+'/*.mpr')[-1]
datfiles = glob.glob(folder+'/*.dat')

In [5]:
ecfdf, ecf_start = read_ecf(ecfile)

In [6]:
joined_emsi = pd.DataFrame([])
for filename in datfiles:
    emsidf, emsi_start, emsi_end = read_emsi(filename)
    if emsi_end < ecf_start:
        continue
    emsi_shift, scale = optimise_emsi_start(ecfdf, ecf_start, emsidf, emsi_start)
    emsidf.loc[:, 't'] = emsidf.t + emsi_start - ecf_start - emsi_shift
    emsidf.loc[:, 'I'] = emsidf.I * scale
    joined_emsi = pd.concat([joined_emsi, emsidf], ignore_index=True)
plt.plot(joined_emsi.t, joined_emsi.I)
plt.plot(ecfdf.t, ecfdf.I)

0.3269720853157294 1001.572248051529
-0.1129665248864082 1000.4131097813929
-0.09475302696315266 998.7135120157802
0.125486054588153 996.0984076176908
0.44696899063825907 998.6440453202513
0.9414417743688934 990.5262818024553
0.3635224127107787 997.425979312564


In [7]:
ecfdf['emsi'] = np.interp(ecfdf.t, joined_emsi.t, joined_emsi.emsi)
ecfdf['light'] = np.interp(ecfdf.t, joined_emsi.t, joined_emsi.light)

In [8]:
ecfdf.to_pickle(folder+'/data.pkl')